# Desafio 1

Conforme definido em sala, os alunos deverão realizar o seguinte desafio pontuado:

- Escolher uma base de dados na área de saúde, educação, segurança pública ou gestão. OBS: o tema dos alunos que faltaram é GESTÃO;
- Criar e enviar o modelo conceitual de Banco de dados;
- Criar o modelo lógico e físico de Banco de Dados;
- Criar o Schema e persistir no Banco de Dados;
- Escolher o dataset, o mesmo deve ser tratado e poderá ter em torno de 1000 linhas (você pode abrir no excel fazer a exclusão de linhas e colunas para ele ficar menor);
- Persistir os dados no banco de dados;
- Fazer todo o processo de CRUD utilizando python;
- Gerar 3 gráficos relacionados ao dataset.
Você deve enviar o dataset tratado, o modelo conceitual, o schema do banco, o arquivo python.

Base de dados: Internações hospitalares no Brasil do mês de dezembro/2024. O dataset, originalmente com mais de um milhão de entradas e cem colunas, foi reduzido a 50000 entradas e 11 colunas.

https://www.kaggle.com/datasets/andersonfranca/sistema-de-informaes-hospitalares-sus/data

Auxiliarmente, foi utilizado também um dataset simples, contendo o código e a descrição das subcategorias do CID10.

http://www2.datasus.gov.br/cid10/V2008/descrcsv.htm#:~:text=CID-10-SUBCATEGORIAS.,CID-O-GRUPOS.

In [17]:
import pandas as pd

df = pd.read_csv('data/inter_hosp_br_202412.csv', sep=';')

df.shape

(50000, 12)

In [18]:
df.columns

Index(['NASC', 'SEXO', 'QT_DIARIAS', 'DIAG_PRINC', 'IDADE', 'DIAS_PERM',
       'MORTE', 'INSTRU', 'RACA_COR', 'CAR_INT', 'COBRANCA', 'ID'],
      dtype='object')

In [19]:
df.dtypes

NASC           int64
SEXO           int64
QT_DIARIAS     int64
DIAG_PRINC    object
IDADE          int64
DIAS_PERM      int64
MORTE          int64
INSTRU         int64
RACA_COR       int64
CAR_INT        int64
COBRANCA       int64
ID             int64
dtype: object

NASC DATA NASCIMENTO NO FORMATO NUMERICO YYYYMMDD

----

QT_DIARIAS EM DIAS

---

IDADE: EM ANOS

---
Sexo
1 : Masculino
2 : Feminino
3 : Feminino
0;9 : NA

In [22]:
df_cid = pd.read_csv('data/CID-10-SUBCATEGORIAS2.csv', encoding='ISO-8859-1')
df_cid.head()

,SUBCAT,DESCRICAO
0,A000,Cólera devida a Vibrio cholerae 01
1,A001,Cólera devida a Vibrio cholerae 01
2,A009,Cólera não especificada
3,A010,Febre tifóide
4,A011,Febre paratifóide A


In [14]:
import matplotlib.pyplot as plt
from sqlalchemy import create_engine, inspect, text

In [15]:
# Dados do banco

user = "ufrkpl872bmu15"
password = "pfc2f800f1a232ad1055c52d4cbc53018d57166e9190b5215ce20c656fd97ed3b"
host = "cbec45869p4jbu.cluster-czrs8kj4isg7.us-east-1.rds.amazonaws.com"
port = "5432"
database = "d487kabe2ju107"

# String de conexão
DATABASE_URL = f"postgresql://{user}:{password}@{host}:{port}/{database}"

# Criação do engine
engine = create_engine(DATABASE_URL)

# Testar conexão (opcional)
with engine.connect() as connection:
    result = connection.execute(text("SELECT 1"))  # <-- corrigido aqui
    print(result.scalar())

1
